<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup-Data-Pre-Processing" data-toc-modified-id="Setup-Data-Pre-Processing-0"><span class="toc-item-num">0&nbsp;&nbsp;</span>Setup Data Pre-Processing</a></span><ul class="toc-item"><li><span><a href="#Raw-Data-to-Feature-Data" data-toc-modified-id="Raw-Data-to-Feature-Data-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Raw Data to Feature Data</a></span></li><li><span><a href="#Tensor-Data-Structure-Input" data-toc-modified-id="Tensor-Data-Structure-Input-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Tensor Data-Structure Input</a></span></li></ul></li><li><span><a href="#Architecture-Basics" data-toc-modified-id="Architecture-Basics-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Architecture Basics</a></span><ul class="toc-item"><li><span><a href="#Gated-CNN" data-toc-modified-id="Gated-CNN-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Gated CNN</a></span><ul class="toc-item"><li><span><a href="#GCNN-1D-Time-Series" data-toc-modified-id="GCNN-1D-Time-Series-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>GCNN 1D Time Series</a></span></li><li><span><a href="#GCNN-1D-Residuals" data-toc-modified-id="GCNN-1D-Residuals-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>GCNN 1D Residuals</a></span></li><li><span><a href="#GCNN-2D" data-toc-modified-id="GCNN-2D-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>GCNN 2D</a></span></li></ul></li></ul></li></ul></div>

In [10]:
# Imports e definicoes auxiliares, TODO - fazer module apenas para estas definicoes auxiliares
from IPython.display import Math
#!pip install librosa
import tensorflow as tf
import numpy as np
import scipy as scp
from scipy import signal, linalg
from numpy import asarray, array, ravel, repeat, prod, mean, where, ones
import librosa
audio_open = librosa.audio.audioread.audio_open

#!pip install scikits.audiolab
#!pip install --upgrade-strategy=only-if-needed git+https://github.com/Uiuran/BregmanToolkit
#!pip install scikit-image
#!pip install --upgrade-strategy=only-if-needed git+https://github.com/librosa/

import matplotlib.pyplot as plt
from tensorflow.python.client import timeline
correlate = signal.correlate
import os

# Tensorboard display
from IPython.display import clear_output, Image, display, HTML
'''
GATED
CONVOLUTIONAL NEURAL NETWORK Architecture P.O.C (proof of concept)
'''

# Change esdruxulo names given by tensorflow development team
tf.variable_scope = tf.compat.v1.variable_scope

# funções auxiliares
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = tf.compat.as_bytes("<stripped %d bytes>"%size)
    return strip_def
  
def rename_nodes(graph_def, rename_func):
    res_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = res_def.node.add() 
        n.MergeFrom(n0)
        n.name = rename_func(n.name)
        for i, s in enumerate(n.input):
            n.input[i] = rename_func(s) if s[0]!='^' else '^'+rename_func(s[1:])
    return res_def
  
# Função que usa HTML e javascript para exibir tensorboar no notebook e web
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
    
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.    
    
    Parameters:
    
    - session: The TensorFlow session to be frozen.
    
    Default Keyword Parameters
    
    - keep_var_names: A list of variable names that should not be frozen. Defaults None.     
    - output_names: Names of the relevant graph outputs/operation/tensor to be written. Defaults None.
    - clear_devices: Remove the device directives from the graph for better portability. Defaults True.
    
    return The frozen graph definition.
    """
    
    graph = session.graph
    with graph.as_default():
      freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
      output_names = output_names or []
      output_names += [v.op.name for v in tf.global_variables()]
      # Graph -> GraphDef ProtoBuf
      input_graph_def = graph.as_graph_def()
      if clear_devices:
        for node in input_graph_def.node:
          node.device = ""
      frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                    output_names, freeze_var_names)
      return frozen_graph
    
def save_model(sess, model_name='./model.pb', output_names=True):
    
  frozen_graph = freeze_session(sess, output_names= output_names)
  tf.train.write_graph(frozen_graph, "model", model_name, as_text=False)   
    
def downsampling(data,num):
  '''
   Downsampling por num(int) feito como se fosse no C
  '''
  d = np.copy(data)
  j=0
  for i in range(len(d)):

    if i%int(num) == 0:
      j +=1
    else:
      d[i] = 0.0
  datadown = np.zeros(j)
  j=0
  for i in range(len(d)):
  
    if d[i] != 0.0: 
      #print(d[i])
      datadown[j] = d[i]
      j += 1  

  return datadown

def amostragem(data, fs, plot = False):
  '''
   Amostragem de downsampling de uma serie temporal segundo Nyquist-Shannon. Alem disso o numero maximo
   de frequencias com o spectro calculado por ciclo deve ser igual ao numero de pontos N/2 (principio de incerteza).

   
   argumentos:
   
   - data: serie temporal
   - fs: frequência de amostragem do encode original
   - plot: opcional, plota o espectrograma com parametros padrões para obter máxima frequência maior que a largura de banda.
  '''
 
  f,t,Sxx = signal.spectrogram(data,fs,nfft=fs/4,nperseg=fs/5,noverlap=fs/10,scaling='spectrum',mode='magnitude')
  freq = np.array([i for i in zip(t,Sxx.T) if np.shape(f[i[1]>np.std(i[1])])[0] != 0])

  if plot:
    plt.figure()
    plt.pcolormesh(t, f, np.log(Sxx+1e-13) )
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()

  return [[ i[0], np.ceil(np.float(fs)/(2.0*np.max(f[i[1]>np.std(i[1])]) )) ] for i in freq]

def subbed_timeseries(measure,fs):
  '''
   Return Sxx (log squared amplitude of stft spectra), f (frequencies), t (time steps)
  '''
  sampa = np.min(np.array(amostragem(np.array(measure),fs, plot=False))[:,1])  
  fsa = (fs/int(sampa))
    
  a = downsampling(np.array(measure),sampa )

  return a,fsa

def subbed_spect(measure,fs,plot=True):
  '''
   Return Sxx (log squared amplitude of stft spectra), f (frequencies), t (time steps)
  '''
  sampa = np.min(np.array(amostragem(np.array(measure),fs, plot=plot))[:,1])  
  fsa = (fs/int(sampa))
    
  a = downsampling(np.array(measure),sampa )
  f,t,Sxx = signal.spectrogram(a,fsa,nfft=fsa/4,nperseg=fsa/5,noverlap=fsa/10,scaling='spectrum',mode='magnitude')
  #Cut zero variance part of spectra(useless recording)   
  Sxx = Sxx[:,np.var(np.log(Sxx+1e-13),axis=0) > 0.5]
  l = len(Sxx[0,:])
  t = t[:l] 
  
  if plot:
    plt.figure()
    plt.pcolormesh(t, f, np.log(Sxx+1e-13))
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()
    
  return np.abs(np.log(Sxx+1e-13)),f,t
    
def sample_gaussmix(data=None,shape=(500,), mean_base = 0.0,var_base = 0.15, num_mods = 30, random_mods=False):
  '''
    Generate sample noise from num_mods different gaussian with data or only noise.
    
    TODO: select mod from a list of tuples with (mean,variance)
  '''
  num_mods = int(num_mods)
  if type(data) != type(None):
    shape = np.shape(data)  
  c = np.int(shape[0]/num_mods)
  np.random.set_state = 1440*shape[0]

  if type(data) != type(None):
    if random_mods:
      if len(shape) > 1:
        mix = lambda data: (data + np.array([np.random.normal(mean_base*np.mod(np.random.randint((i+1),high=1234*i+4),num_mods),var_base*np.mod(np.random.randint((i+1),high=1234*i+4),num_mods),(c,shape[1:]) ) for i in range(num_mods)]).flatten() )
      else:
        mix = lambda data: (data + np.array([np.random.normal(mean_base*np.mod(np.random.randint((i+1),high=1234*i+4),num_mods),var_base*np.mod(np.random.randint((i+1),high=1234*i+4),num_mods),c) for i in range(num_mods)]).flatten() )    
    else:
      if len(shape) > 1:
        mix = lambda data: (data + np.array([np.random.normal(mean_base*(i+1),var_base*(i+1),(c,shape[1:]) ) for i in range(num_mods)]).flatten() )
      else:
        mix = lambda data: (data + np.array([np.random.normal(mean_base*(i+1),var_base*(i+1),c) for i in range(num_mods)]).flatten() )      
  else:
    if random_mods:
      if len(shape) > 1:
        mix = lambda data: (np.array([np.random.normal(mean_base*np.mod(np.random.randint((i+1),high=1234*i+4),num_mods),var_base*np.mod(np.random.randint((i+1),high=1234*i+4),num_mods),(c,shape[1:]) ) for i in range(num_mods)]).flatten() )
      else:
        mix = lambda data: (np.array([np.random.normal(mean_base*np.mod(np.random.randint((i+1),high=1234*i+4),num_mods),var_base*np.mod(np.random.randint((i+1),high=1234*i+4),num_mods),c) for i in range(num_mods)]).flatten() )    
    else:
      if len(shape) > 1:
        mix = lambda data: (np.array([np.random.normal(mean_base*(i+1),var_base*(i+1),(c,shape[1:]) ) for i in range(num_mods)]).flatten() )
      else:
        mix = lambda data: (np.array([np.random.normal(mean_base*(i+1),var_base*(i+1),c) for i in range(num_mods)]).flatten() )      
  return mix(data)

#Device function helpers

class _dev_selector:
    
  def __init__(self,**kwargs):
    self.arg1 = kwargs['arg1']
    
  def __call__(self,op):
    if op.name.split('/')[0] == 'gcnn2d_d1':
      return '/device:CPU:0'
    elif op.name.split('/')[0] == 'gcnn2d':
      return '/device:CPU:0'
    else:
      return '/device:CPU:0'

  def _conv2D_on_gpu(op):
    if op.type == "Conv2D":
      return "/device:GPU:*"
    else:
      return "/device:CPU:*"

    
# In TensorFlow a device name is a string of the following form:
#   /job:<name>/replica:<replica>/task:<task>/device:<type>:<device_num>
#
# <name> is a short identifier conforming to the regexp
#     [a-zA-Z][_a-zA-Z]*
# <type> is a supported device type (e.g. 'cpu' or 'gpu')
# <replica>, <task>, <device_num> are small non-negative integers and are
# densely allocated (except in tests).
#
# For some purposes, we also allow device patterns, which can specify
# some or none of the specific fields above, with missing components,
# or "<component>:*" indicating "any value allowed for that component.
#
# For example:
#   "/job:param_server"   - Consider any devices in the "param_server" job
#   "/device:cpu:*"       - Consider any cpu devices in any job/task/replica
#   "/job:*/replica:*/task:*/device:cpu:*"  - Consider any cpu devices in any
#                                             job/task/replica
#   "/job:w/replica:0/task:0/device:gpu:*"  - Consider any gpu devices in
#                                             replica 0, task 0, of job "w"

def print_display_cgraph(graph):
  for op in graph.get_operations():
    print(op.name)
    graph_def = graph.as_graph_def()
  show_graph(graph_def)

# Setup Data Pre-Processing 

## Raw Data to Feature Data

In [79]:
# Raw data to Log-Spectogram 
# Read data from the hard-drive
spect_list = []
raw_list = []
nomes = [f for f in os.listdir('/home/stattus4dpenalva/stattus4/src/')]
label_list = []

for i in range(len(nomes[0:10])):
  #ignore .mat
  if nomes[i].find('.mat') == -1:    
    data,fs = librosa.load('/home/stattus4dpenalva/stattus4/src/'+nomes[i],sr=None)
    spect_list.append( subbed_spect(data,fs,plot=False) )
    raw_list.append((data,fs))
    label_list.append(nomes[i][0:2])

## Tensor Data-Structure Input

In [86]:
# Data to Tensor Data for consumption on Tensorflow models
# TODO - equalize data size for downsampled raw data
# - On supposition that the data size is the same for all time series, if not is necessary to assert it.
# - The same is true for spect, however downsampled spect may lead to smaller ts.
#ts  = sample_gaussmix(shape=(10000,), mean_base=0.5, var_base=0.15, random_mods=True, num_mods=50)

batch = len(spect_list)
data_size_ts = len(raw_list[0][0])
data_size_image = (len(spect_list[0][1]),len(spect_list[0][2]))
channels = 1

data_tensor_1D = np.ndarray( shape=(batch,data_size_ts,channels) )
data_tensor_2D = np.ndarray( shape=(batch,data_size_image[0],data_size_image[1],channels) )

#for i in range(batch):
  #data_tensor_1D[i,:,0] = raw_list[i][0]
  #data_tensor_2D[i,:,:,0] = spect_list[i][0][:,:]

# Architecture Basics

In [47]:
# Definitions: Build computation graph and run session
def build_graph(data_shape, arch = 'gcnn1d', show_cgraph = True, new_graph = True, name_scope = True, **kwargs):
  '''
    Build a Tensorflow computational graph.
    Currently supports 1-D gcnn or residual 1-D gcnn for unidimensional variable and 2D gcnn for bidimensional frame data.
        
    kwargs:
    filter_size: conv filter size. Scalar number for time series, bidimensional tuple for image.
    channels_out: Number of channels for output convolution.
    
    TODO - export serialized file containing the model (e.g. .pb)
  '''

  if kwargs:
    filter_size = kwargs['filter_size']
    channels_out = kwargs['channels_out']
  else:
    if arch == 'gcnn1d' or arch == 'residual gcnn1d':
      filter_size = 8000
      channels_out = 4
        
    if arch == 'gcnn2d':
      filter_size = (40,2)
      channels_out = 128
    
  if arch == 'gcnn2d':    
    # assert existence of the graph  
    try: 
      assert type(tf.get_default_graph()) == type(tf.Graph())
      graph = tf.get_default_graph()
    except AssertionError:
      graph = tf.Graph()
    
    if new_graph:
      tf.reset_default_graph()
    
    if name_scope:
        
      with graph.as_default():
        with tf.variable_scope(arch):        
          signal_in = tf.placeholder(tf.float32,(data_shape[0],data_shape[1],data_shape[2],data_shape[3]), name='signal_in')
          ####        
          ## Keras convolutions. Classes, so dont behave like functions but outputs Tensor, use its functions to query the variables filters and bias
      
          with graph.device(_dev_selector(arg1='foo')):            
            conv_linear = tf.keras.layers.Conv2D( channels_out, filter_size, padding='valid', name='conv_linear', use_bias=True, kernel_initializer=tf.initializers.lecun_normal(seed=137), bias_initializer=tf.initializers.lecun_normal(seed=137) )(signal_in)

          with graph.device(_dev_selector(arg1='foo')):
            conv_gate = tf.sigmoid(tf.keras.layers.Conv2D( channels_out, filter_size, padding='valid', name='conv', use_bias=True,  kernel_initializer=tf.initializers.lecun_normal(seed=137), bias_initializer=tf.initializers.lecun_normal(seed=137) )(signal_in),name='conv_sigmoid')

          with graph.device(_dev_selector(arg1='foo')):
            gated_convolutions = tf.multiply(conv_linear,conv_gate,name='gated_convolutions')
          #probability = tf.nn.softmax(gated_convolutions, axis=None, name='probability')
          #minimize_op = tf.train.AdamOptimizer(learning_rate=0.005).minimize(-loss)            
        #tf.global_variables_initializer()
        
          if show_cgraph:
            print_display_cgraph(graph)
            
    else:
        
      with graph.as_default():
        signal_in = tf.placeholder(tf.float32,(data_shape[0],data_shape[1],data_shape[2],data_shape[3]), name='signal_in')
        ####        
        ## Keras convolutions. Classes, so dont behave like functions but outputs Tensor, use its functions to query the variables filters and bias
        
        with graph.device(_dev_selector(arg1='foo')):
          conv_linear = tf.keras.layers.Conv2D( channels_out, filter_size, padding='valid', name='conv_linear', use_bias=True,  kernel_initializer=tf.initializers.lecun_normal(seed=137), bias_initializer=tf.initializers.lecun_normal(seed=137) )(signal_in)

        with graph.device(_dev_selector(arg1='foo')):
          conv_gate = tf.sigmoid(tf.keras.layers.Conv2D( channels_out, filter_size, padding='valid', name='conv', use_bias=True,  kernel_initializer=tf.initializers.lecun_normal(seed=137), bias_initializer=tf.initializers.lecun_normal(seed=137)  )(signal_in),name='conv_sigmoid')
        
        with graph.device(_dev_selector(arg1='foo')):
          gated_convolutions = tf.multiply(conv_linear,conv_gate,name='gated_convolutions')
        #probability = tf.nn.softmax(gated_convolutions, axis=None, name='probability')
        #minimize_op = tf.train.AdamOptimizer(learning_rate=0.005).minimize(-loss)
        
        if show_cgraph:
          print_display_cgraph(graph)

  if arch == 'gcnn1d':    
    # assert existence of the graph  
    try: 
      assert type(tf.get_default_graph()) == type(tf.Graph())
      graph = tf.get_default_graph()
    except AssertionError:
      graph = tf.Graph()
    
    if new_graph:
      tf.reset_default_graph()

    with graph.as_default():
      signal_in = tf.placeholder(tf.float32,(data_shape[0],data_shape[1],data_shape[2]), name='signal_in')
      ####
      ## Keras convolutions. Classes, so dont behave like functions but outputs Tensor, use its functions to query the variables filters and bias
      conv_linear = tf.keras.layers.Conv1D( channels_out, filter_size, padding='causal', name='conv_linear', use_bias=True)(signal_in)
      conv_gate = tf.sigmoid(tf.keras.layers.Conv1D( channels_out, filter_size, padding='causal', name='conv', use_bias=True )(signal_in),name='conv_sigmoid')
      gated_convolutions = tf.multiply(conv_linear,conv_gate,name='gated_convolutions')
        
      #probability = tf.nn.softmax(gated_convolutions, axis=None, name='probability')
      #minimize_op = tf.train.AdamOptimizer(learning_rate=0.005).minimize(-loss)        
        
      if show_cgraph:  
        print_display_cgraph(graph)
        
  if arch == 'residual gcnn1d':
    
    try: 
      assert type(tf.get_default_graph()) == type(tf.Graph())
      graph = tf.get_default_graph()
    except AssertionError:
      graph = tf.Graph()
    
    if new_graph:
      tf.reset_default_graph()
    
    with graph.as_default():
        
      signal_in = tf.placeholder(tf.float32,(data_shape[0],data_shape[1],data_shape[2]), name='signal_in')
    
      ####
      ## Keras convolutions. Classes, so dont behave like functions but outputs Tensor, use its functions to query the variables filters and bias
      conv_linear = tf.keras.layers.Conv1D( channels_out, filter_size, padding='causal', name='conv_linear', use_bias=True)(signal_in)
      conv_gate = tf.sigmoid(tf.keras.layers.Conv1D( channels_out, filter_size, padding='causal', name='conv', use_bias=True )(signal_in),name='conv_sigmoid')
      gated_convolutions = tf.multiply(conv_linear,conv_gate,name='gated_convolutions')
      # Input channels must be the same size of the convolution channels output (ie number of filters applied)
      residual = tf.add(gated_convolutions,signal_in,name='residual')                
    
      if show_cgraph:  
        print_display_cgraph(graph)
    
  return graph

# Definitions: Build computation graph and run session
def build_graph_module(graph, scope_tensor_name=None, arch = 'gcnn1d',print_ops = True, name_scope = True, show_cgraph = True, data_shape = (None,40,2,1), **kwargs):
  '''
    Build a Tensorflow computational graph.building_function
    Currently supports 1-D gcnn or residual 1-D gcnn for unidimensional variable and 2D gcnn for bidimensional frame data.
    
    tensor_scope_name:  name scope of the architecture unit to build the next module
    
        
    kwargs:    
    filter_size: conv filter size. Scalar number for time series, bidimensional tuple for image.
    channels_out: Number of channels for output convolution.
    deepness: howdeep is the network
    
    TODO - export serialized file containing the model (e.g. .pb)
  '''
  
  if kwargs:
    filter_size = kwargs['filter_size']
    channels_out = kwargs['channels_out']
    deepness = kwargs['deepness']
  else:
    if arch == 'gcnn1d' or arch == 'residual gcnn1d':
      filter_size = 8000
      channels_out = 4
      deepness = ''  
    if arch == 'gcnn2d':
      filter_size = (40,2)
      channels_out = 128
      deepness = ''  
    
  if arch == 'gcnn2d':       

      ####        
      ## Keras convolutions. Classes, so dont behave like functions but outputs Tensor, use its functions to query the variables filters and bias    

    if name_scope:
      
      with graph.as_default():            
        with tf.variable_scope(arch+deepness):        
          if type(scope_tensor_name) == type(None):
            get_scope_tensor = tf.placeholder(tf.float32,(data_shape[0],data_shape[1],data_shape[2],data_shape[3]), name='signal_in')
          else:        
            get_scope_tensor = graph.get_tensor_by_name(scope_tensor_name+':0') 
            
          ####        
          ## Keras convolutions. Classes, so dont behave like functions but outputs Tensor, use its functions to query the variables filters and bias   
          channelsout = channels_out
          # TODO filter_size must be less than (input size + dilate)
          filtersize = (filter_size[0],filter_size[1])
          with graph.device(_dev_selector(arg1='foo')):
            conv_linear = tf.keras.layers.Conv2D( channelsout, filtersize, padding='valid', name='conv_linear', use_bias=True,  kernel_initializer=tf.initializers.lecun_normal(seed=137), bias_initializer=tf.initializers.lecun_normal(seed=137) )(get_scope_tensor)
            
          with graph.device(_dev_selector(arg1='foo')):
            conv_gate = tf.sigmoid(tf.keras.layers.Conv2D( channelsout, filter_size, padding='valid', name='conv', use_bias=True,  kernel_initializer=tf.initializers.lecun_normal(seed=137), bias_initializer=tf.initializers.lecun_normal(seed=137) )(get_scope_tensor),name='conv_sigmoid')
          
          with graph.device(_dev_selector(arg1='foo')):
            gated_convolutions = tf.multiply(conv_linear,conv_gate,name='gated_convolutions')
            #probability = tf.nn.softmax(gated_convolutions, axis=None, name='probability')
            #minimize_op = tf.train.AdamOptimizer(learning_rate=0.005).minimize(-loss)
        

          if show_cgraph:
            print_display_cgraph(graph)
            
    else:
        
      with graph.as_default():
        
        if type(scope_tensor_name) == type(None):
          get_scope_tensor = tf.placeholder(tf.float32,(data_shape[0],data_shape[1],data_shape[2],data_shape[3]), name='signal_in')
        else:        
          get_scope_tensor = graph.get_tensor_by_name(scope_tensor_name+':0')        
        
        channelsout = channels_out
        # TODO filter_size must be less than (input size + dilate)
        filtersize = (filter_size[0],filter_size[1])                
        ####        
        ## Keras convolutions. Classes, so dont behave like functions but outputs Tensor, use its functions to query the variables filters and bias      
        with graph.device(_dev_selector(arg1='foo')):
          conv_linear = tf.keras.layers.Conv2D( channels_out, filter_size, padding='valid', name='conv_linear', use_bias=True,  kernel_initializer=tf.initializers.lecun_normal(seed=137), bias_initializer=tf.initializers.lecun_normal(seed=137) )(get_scope_tensor)
        with graph.device(_dev_selector(arg1='foo')):
          conv_gate = tf.sigmoid(tf.keras.layers.Conv2D( channels_out, filter_size, padding='valid', name='conv', use_bias=True,  kernel_initializer=tf.initializers.lecun_normal(seed=137), bias_initializer=tf.initializers.lecun_normal(seed=137)  )(get_scope_tensor),name='conv_sigmoid')
        with graph.device(_dev_selector(arg1='foo')):
          gated_convolutions = tf.multiply(conv_linear,conv_gate,name='gated_convolutions')
          #probability = tf.nn.softmax(gated_convolutions, axis=None, name='probability')
          #minimize_op = tf.train.AdamOptimizer(learning_rate=0.005).minimize(-loss)
        
        if show_cgraph:
          print_display_cgraph(graph)

  if arch == 'gcnn1d':    
    # assert existence of the graph      
    with graph.as_default():
      
      ####
      ## Keras convolutions. Classes, so dont behave like functions but outputs Tensor, use its functions to query the variables filters and bias
    
      if type(tensor_name) == type(None):
        get_tensor = graph.get_tensor_by_name(graph.get_operations()[-1].name+':0')
      else:
        get_tensor = graph.get_tensor_by_name(tensor_name+':0')
        
      conv_linear = tf.keras.layers.Conv1D( channels_out, filter_size, padding='causal', name='conv_linear', use_bias=True)(get_tensor)
      conv_gate = tf.sigmoid(tf.keras.layers.Conv1D( channels_out, filter_size, padding='causal', name='conv', use_bias=True )(get_tensor),name='conv_sigmoid')
      gated_convolutions = tf.multiply(conv_linear,conv_gate,name='gated_convolutions')
      #probability = tf.nn.softmax(gated_convolutions, axis=None, name='probability')
      #minimize_op = tf.train.AdamOptimizer(learning_rate=0.005).minimize(-loss)
        
      if show_cgraph:
        print_display_cgraph(graph)
        
  if arch == 'residual gcnn1d':
    
    with graph.as_default():        
  
      ####
      ## Keras convolutions. Classes, so dont behave like functions but outputs Tensor, use its functions to query the variables filters and bias
        
      if type(tensor_name) == type(None):
        get_tensor = graph.get_tensor_by_name(graph.get_operations()[-1].name+':0')
      else:
        get_tensor = graph.get_tensor_by_name(tensor_name+':0')
        
      conv_linear = tf.keras.layers.Conv1D( channels_out, filter_size, padding='causal', name='conv_linear', use_bias=True)(get_tensor)
      conv_gate = tf.sigmoid(tf.keras.layers.Conv1D( channels_out, filter_size, padding='causal', name='conv', use_bias=True )(get_tensor),name='conv_sigmoid')
      gated_convolutions = tf.multiply(conv_linear,conv_gate,name='gated_convolutions')
      # Input channels must be the same size of the convolution channels output (ie number of filters applied)
      residual = tf.add(gated_convolutions,signal_in,name='residual')
        
      if show_cgraph:
        print_display_cgraph(graph)
        
  return graph
        
def run_cgraph(graph, op_to_run, feed_dict, number_of_runs = 10, mode = 'feed forward', new_session=True, output_log = True, verbose=True):
  '''
     Run Session
     TODO - session log output
     TODO - in the mode feed forward, only get tensor value for input, in the mode minimize run the last op that represents minimization     
     TODO - make op to run optional depending on modes
     TODO - getvariable by namescope
  '''
    # assert existence of the session and graph         
  assert type(graph) == type(tf.Graph())   
       
  if new_session:
    session = tf.Session(graph=graph)  
  else:
    session = tf.get_default_session()
    
  with session.as_default():
    #with tf.device(_dev_selector(''))
    feed_dict = feed_dict
    options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    opval = np.zeros(number_of_runs)
    
    # Search for Initialization: TODO binary search, using brute force ... 
    ops = graph.get_operations()
    opinit = ops[-1]
    if opinit.name[-4:] != 'init' and str(opinit.type) != 'NoOp':
      c = 0  
      for op in ops:
        if op.name[-4:] == 'init' and str(op.type) == 'NoOp':
          session.run(op)
          print('Modulo Variables Initialized {}'.format(c) )
          c += 1
      if c == 0:
        with graph.as_default():
          tf.global_variables_initializer()
          session.run(graph.get_operations()[-1])
          print('Global Variables Initialized')
    
    else:
      session.run(opinit)
      print('Global Variables Initialized')
        
    print_display_cgraph(graph)
    # Perform  gradient descent steps or feed forward according to the Operation Required   
    for step in range(number_of_runs):
    
      op_value = session.run(op_to_run, feed_dict=feed_dict)        
      #opval[step] = op_value  
      if step % 1 == 0 and verbose:        
        print("Step:", step, " Loss:", op_value)
        if step % 5 == 0 and step != 0:          
          loss_diff = np.diff(opval[np.nonzero(opval)])
          print("Mean Loss Growth ", np.mean(loss_diff) )    

      #gated_convs_value = session.run(graph.get_tensor_by_name('gated_convolutions:0'), feed_dict = feed_dict, options=options, run_metadata=run_metadata)
  return op_value

##  Gated CNN
  Gated CNN is a doubled CNN in whom one of the convoluted signals does the role of opening/closing the network, giving an **Attention Mechanism** to the convolution, for being activated by a sigmoid.
  It gives non-vanishing gradient, since the multiplication rule for the derivative applies, also, applies gradient to the linear convoluted part.
  

### GCNN 1D Time Series
  In time series version, since the desirable learning is based on **past** events, or you cannot uphold the assumption that you have acess to future data, have to make sure that the convolution is **causal**, that is
\begin{align}
y_{n}= a_{i}x_{n-i}=a_{n-j}x_{j}
\end{align} 
  Giving at last, if the filter has length k, k-1 zero padding to the input x.

In [28]:
#build graph
graph = build_graph( (None,l[0],channels), arch = 'gcnn1d',print_ops = True, new_graph=True, show_cgraph = True)
'''
obs: if you have a graph sometimes is necessary to run more than one time wit reset_default_graph to get a new graph
'''

NameError: name 'l' is not defined

In [96]:
# Run the constructed graph

feed_dict = {
  graph.get_tensor_by_name(graph.get_operations()[-1].name+':0'):data_tensor
}

op_value = run_cgraph(graph, graph.get_operations()[-1].name+':0', feed_dict, number_of_runs = 10, mode = 'feed forward', new_session=True, output_log = True, verbose=True)
print(op_value)

('Step:', 0, ' Loss:', array([[[ 0.7076101 ,  0.75356215,  0.44474468,  0.89668995],
        [ 0.404285  ,  0.10061911,  0.8874869 ,  0.22097981],
        [-0.05257223,  0.8174497 ,  0.00637231,  0.00501409],
        ...,
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ]]],
      dtype=float32))
('Step:', 1, ' Loss:', array([[[ 0.7076101 ,  0.75356215,  0.44474468,  0.89668995],
        [ 0.404285  ,  0.10061911,  0.8874869 ,  0.22097981],
        [-0.05257223,  0.8174497 ,

/home/stattus4dpenalva/miniconda2/envs/scicomptf_37/lib/python2.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/stattus4dpenalva/miniconda2/envs/scicomptf_37/lib/python2.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


### GCNN 1D Residuals

In [32]:
#build graph
graph = build_graph( (None,l[0],channels), arch = 'residual gcnn1d',print_ops = True,new_graph=True, show_cgraph = True)

NameError: name 'l' is not defined

In [31]:
# Run the constructed graph

feed_dict = {
  graph.get_tensor_by_name(graph.get_operations()[-1].name+':0'):data_tensor
}

op_value = run_cgraph(graph, graph.get_operations()[-1].name+':0', feed_dict, number_of_runs = 10, mode = 'feed forward', new_session=True, output_log = True, verbose=True)
print(op_value)

KeyError: "The name 'init:0' refers to a Tensor which does not exist. The operation, 'init', exists but only has 0 outputs."

### GCNN 2D
  This unit uses sigmoid function in all frame pixels and do element-wise multiplication. Use it for images or time-series TF Features frame. The standard convolution mode is Valid. The output size of the convolution is given by:
 
 Padding Valid
\begin{align}
dimemsion output size = \ceil{ \frac {dim size - (kernel dim size - 1)*dilation rate}{stride} }
\end{align} 

Where dilation rate is how may pixel are each filter spaced from each other, when dilation rate >= 1 then stride = 1 (since if you stride will be losing information from the data).

In [48]:
#build graph
frame_height = 40 # put number of FT features for 2D conv
frame_width = 2 # here is the same, will depend on fft size and window
channels = 1

graph = build_graph( (None,frame_height,frame_width,channels), arch = 'gcnn2d',print_ops = True,new_graph=True, show_cgraph = True, filter_size=(2,2), channels_out = 32)
op = graph.get_operations()[-1]

#with tf.Session('grpc://192.168.100.122:2222',graph=graph) as sess:
#  for _ in range(10000):
#    sess.run(op, feed_dict=)
    
graph = build_graph_module(graph, scope_tensor_name=op.name, arch = 'gcnn2d',print_ops = True, name_scope = True, show_cgraph = True, filter_size=(2,1), channels_out = 64, deepness = '_d1')
graph = build_graph_module(graph, scope_tensor_name=None, arch = 'gcnn2d',print_ops = True, name_scope = True, show_cgraph = True, filter_size=(2,1), channels_out = 64, deepness = '')
op = graph.get_operations()[-1]
graph = build_graph_module(graph, scope_tensor_name=op.name, arch = 'gcnn2d',print_ops = True, name_scope = True, show_cgraph = True, filter_size=(2,1), channels_out = 64, deepness = '_d1')

gcnn2d/signal_in
gcnn2d/conv_linear/kernel/Initializer/truncated_normal/shape
gcnn2d/conv_linear/kernel/Initializer/truncated_normal/mean
gcnn2d/conv_linear/kernel/Initializer/truncated_normal/stddev
gcnn2d/conv_linear/kernel/Initializer/truncated_normal/TruncatedNormal
gcnn2d/conv_linear/kernel/Initializer/truncated_normal/mul
gcnn2d/conv_linear/kernel/Initializer/truncated_normal
gcnn2d/conv_linear/kernel
gcnn2d/conv_linear/kernel/IsInitialized/VarIsInitializedOp
gcnn2d/conv_linear/kernel/Assign
gcnn2d/conv_linear/kernel/Read/ReadVariableOp
gcnn2d/conv_linear/bias/Initializer/truncated_normal/shape
gcnn2d/conv_linear/bias/Initializer/truncated_normal/mean
gcnn2d/conv_linear/bias/Initializer/truncated_normal/stddev
gcnn2d/conv_linear/bias/Initializer/truncated_normal/TruncatedNormal
gcnn2d/conv_linear/bias/Initializer/truncated_normal/mul
gcnn2d/conv_linear/bias/Initializer/truncated_normal
gcnn2d/conv_linear/bias
gcnn2d/conv_linear/bias/IsInitialized/VarIsInitializedOp
gcnn2d/conv_li

gcnn2d/signal_in
gcnn2d/conv_linear/kernel/Initializer/truncated_normal/shape
gcnn2d/conv_linear/kernel/Initializer/truncated_normal/mean
gcnn2d/conv_linear/kernel/Initializer/truncated_normal/stddev
gcnn2d/conv_linear/kernel/Initializer/truncated_normal/TruncatedNormal
gcnn2d/conv_linear/kernel/Initializer/truncated_normal/mul
gcnn2d/conv_linear/kernel/Initializer/truncated_normal
gcnn2d/conv_linear/kernel
gcnn2d/conv_linear/kernel/IsInitialized/VarIsInitializedOp
gcnn2d/conv_linear/kernel/Assign
gcnn2d/conv_linear/kernel/Read/ReadVariableOp
gcnn2d/conv_linear/bias/Initializer/truncated_normal/shape
gcnn2d/conv_linear/bias/Initializer/truncated_normal/mean
gcnn2d/conv_linear/bias/Initializer/truncated_normal/stddev
gcnn2d/conv_linear/bias/Initializer/truncated_normal/TruncatedNormal
gcnn2d/conv_linear/bias/Initializer/truncated_normal/mul
gcnn2d/conv_linear/bias/Initializer/truncated_normal
gcnn2d/conv_linear/bias
gcnn2d/conv_linear/bias/IsInitialized/VarIsInitializedOp
gcnn2d/conv_li

gcnn2d/signal_in
gcnn2d/conv_linear/kernel/Initializer/truncated_normal/shape
gcnn2d/conv_linear/kernel/Initializer/truncated_normal/mean
gcnn2d/conv_linear/kernel/Initializer/truncated_normal/stddev
gcnn2d/conv_linear/kernel/Initializer/truncated_normal/TruncatedNormal
gcnn2d/conv_linear/kernel/Initializer/truncated_normal/mul
gcnn2d/conv_linear/kernel/Initializer/truncated_normal
gcnn2d/conv_linear/kernel
gcnn2d/conv_linear/kernel/IsInitialized/VarIsInitializedOp
gcnn2d/conv_linear/kernel/Assign
gcnn2d/conv_linear/kernel/Read/ReadVariableOp
gcnn2d/conv_linear/bias/Initializer/truncated_normal/shape
gcnn2d/conv_linear/bias/Initializer/truncated_normal/mean
gcnn2d/conv_linear/bias/Initializer/truncated_normal/stddev
gcnn2d/conv_linear/bias/Initializer/truncated_normal/TruncatedNormal
gcnn2d/conv_linear/bias/Initializer/truncated_normal/mul
gcnn2d/conv_linear/bias/Initializer/truncated_normal
gcnn2d/conv_linear/bias
gcnn2d/conv_linear/bias/IsInitialized/VarIsInitializedOp
gcnn2d/conv_li

gcnn2d/signal_in
gcnn2d/conv_linear/kernel/Initializer/truncated_normal/shape
gcnn2d/conv_linear/kernel/Initializer/truncated_normal/mean
gcnn2d/conv_linear/kernel/Initializer/truncated_normal/stddev
gcnn2d/conv_linear/kernel/Initializer/truncated_normal/TruncatedNormal
gcnn2d/conv_linear/kernel/Initializer/truncated_normal/mul
gcnn2d/conv_linear/kernel/Initializer/truncated_normal
gcnn2d/conv_linear/kernel
gcnn2d/conv_linear/kernel/IsInitialized/VarIsInitializedOp
gcnn2d/conv_linear/kernel/Assign
gcnn2d/conv_linear/kernel/Read/ReadVariableOp
gcnn2d/conv_linear/bias/Initializer/truncated_normal/shape
gcnn2d/conv_linear/bias/Initializer/truncated_normal/mean
gcnn2d/conv_linear/bias/Initializer/truncated_normal/stddev
gcnn2d/conv_linear/bias/Initializer/truncated_normal/TruncatedNormal
gcnn2d/conv_linear/bias/Initializer/truncated_normal/mul
gcnn2d/conv_linear/bias/Initializer/truncated_normal
gcnn2d/conv_linear/bias
gcnn2d/conv_linear/bias/IsInitialized/VarIsInitializedOp
gcnn2d/conv_li

gcnn2d_d1_1/conv_linear/bias/Read/ReadVariableOp
gcnn2d_d1_1/conv_linear/dilation_rate
gcnn2d_d1_1/conv_linear/Conv2D/ReadVariableOp
gcnn2d_d1_1/conv_linear/Conv2D
gcnn2d_d1_1/conv_linear/BiasAdd/ReadVariableOp
gcnn2d_d1_1/conv_linear/BiasAdd
gcnn2d_d1_1/conv/kernel/Initializer/truncated_normal/shape
gcnn2d_d1_1/conv/kernel/Initializer/truncated_normal/mean
gcnn2d_d1_1/conv/kernel/Initializer/truncated_normal/stddev
gcnn2d_d1_1/conv/kernel/Initializer/truncated_normal/TruncatedNormal
gcnn2d_d1_1/conv/kernel/Initializer/truncated_normal/mul
gcnn2d_d1_1/conv/kernel/Initializer/truncated_normal
gcnn2d_d1_1/conv/kernel
gcnn2d_d1_1/conv/kernel/IsInitialized/VarIsInitializedOp
gcnn2d_d1_1/conv/kernel/Assign
gcnn2d_d1_1/conv/kernel/Read/ReadVariableOp
gcnn2d_d1_1/conv/bias/Initializer/truncated_normal/shape
gcnn2d_d1_1/conv/bias/Initializer/truncated_normal/mean
gcnn2d_d1_1/conv/bias/Initializer/truncated_normal/stddev
gcnn2d_d1_1/conv/bias/Initializer/truncated_normal/TruncatedNormal
gcnn2d_

In [49]:
# Run the constructed graph
data_tensor = np.random.rand(10,40,2,1)


for op in graph.get_operations():
  if op.name.split('/')[-1] == 'signal_in' and op.name.split('/')[0] == 'gcnn2d_1':

    feed_dict = {
      graph.get_tensor_by_name(op.name+':0'):data_tensor
    }


for op in graph.get_operations():
  if op.name.split('/')[-1] == 'gated_convolutions' and op.name.split('/')[0] == 'gcnn2d_d1_1':
    op_value = run_cgraph(graph, op.name+':0', feed_dict, number_of_runs = 10, mode = 'feed forward', new_session=True, output_log = True, verbose=True)
print(np.shape(op_value) )

Global Variables Initialized
gcnn2d/signal_in
gcnn2d/conv_linear/kernel/Initializer/truncated_normal/shape
gcnn2d/conv_linear/kernel/Initializer/truncated_normal/mean
gcnn2d/conv_linear/kernel/Initializer/truncated_normal/stddev
gcnn2d/conv_linear/kernel/Initializer/truncated_normal/TruncatedNormal
gcnn2d/conv_linear/kernel/Initializer/truncated_normal/mul
gcnn2d/conv_linear/kernel/Initializer/truncated_normal
gcnn2d/conv_linear/kernel
gcnn2d/conv_linear/kernel/IsInitialized/VarIsInitializedOp
gcnn2d/conv_linear/kernel/Assign
gcnn2d/conv_linear/kernel/Read/ReadVariableOp
gcnn2d/conv_linear/bias/Initializer/truncated_normal/shape
gcnn2d/conv_linear/bias/Initializer/truncated_normal/mean
gcnn2d/conv_linear/bias/Initializer/truncated_normal/stddev
gcnn2d/conv_linear/bias/Initializer/truncated_normal/TruncatedNormal
gcnn2d/conv_linear/bias/Initializer/truncated_normal/mul
gcnn2d/conv_linear/bias/Initializer/truncated_normal
gcnn2d/conv_linear/bias
gcnn2d/conv_linear/bias/IsInitialized/VarI

('Step:', 0, ' Loss:', array([[[[ 3.66253495e-01,  9.19417362e-05, -6.35769144e-02, ...,
          -1.70447100e-02,  1.46226317e-01,  1.51670203e-01],
         [ 4.26717311e-01,  8.44392031e-02, -4.51415963e-02, ...,
           3.77011411e-02,  6.50011227e-02, -4.03596722e-02]],

        [[ 1.47952974e-01, -3.46870325e-03,  6.96537644e-03, ...,
           3.77314463e-02,  3.97458635e-02,  2.16283724e-02],
         [ 5.31510949e-01,  5.35677411e-02, -6.14114925e-02, ...,
          -5.93907712e-03,  1.43437892e-01, -6.30669966e-02]],

        [[ 2.57976621e-01,  6.47433847e-03, -1.07914377e-02, ...,
           4.90160845e-02,  1.59486569e-02, -1.73236698e-01],
         [ 3.63499165e-01,  3.66614796e-02, -6.80654794e-02, ...,
           1.49081247e-02,  1.19096637e-01,  2.26006255e-01]],

        ...,

        [[ 4.27812338e-01,  7.23214597e-02, -9.22019035e-02, ...,
           3.18941809e-02,  1.24574497e-01,  3.37028384e-01],
         [ 2.92398930e-01, -2.14446522e-02, -2.73787156e-02, 